In [1]:
# import required libraries
import pandas as pd

In [19]:
# read all the data into dataframes
ken_cust_df = pd.read_csv("data/Kenya Customers.csv")
ken_del_df = pd.read_csv("data/Kenya Deliveries.csv")
ken_ord_df = pd.read_csv("data/Kenya Orders.csv")
nig_cust_df = pd.read_csv("data/Nigeria Customers.csv")
nig_del_df = pd.read_csv("data/Nigeria Deliveries.csv")
nig_ord_df = pd.read_csv("data/Nigeria Orders.csv")

## Merge datasets
1. Get the columns in different 

In [58]:
# Rename columns to match columns in other dataframes
nig_del_df = ken_del_df.rename(columns={"Order_ID": "Order ID"})
ken_del_df = ken_del_df.rename(columns={"Order_ID": "Order ID"})

In [50]:
# Match pattern
pat=r'YR-(\d+),0'
ken_del_df["Order ID"] = ken_del_df["Order ID"].str.replace(pat, r'\1', regex=True)
nig_del_df["Order ID"] = nig_del_df["Order ID"].str.replace(pat, r'\1', regex=True)

In [54]:
# Merge the dataframes for Kenya on common columns
ken_ord_cust_df = pd.merge(ken_cust_df, ken_ord_df, on='Customer ID')
ken_ord_cust_df['Order ID'] = ken_ord_cust_df['Order ID'].astype(str)
ken_df = pd.merge(ken_ord_cust_df, ken_del_df, on='Order ID')
ken_df["Country_Code"] = 254

In [55]:
# Merge the dataframes for Nigeria on common columns
nig_ord_cust_df = pd.merge(nig_cust_df, nig_ord_df, on='Customer ID')
nig_ord_cust_df['Order ID'] = nig_ord_cust_df['Order ID'].astype(str)
nig_df = pd.merge(nig_ord_cust_df, nig_del_df, on='Order ID')
nig_df["Country_Code"] = 234

In [56]:
# Append the Nigeria data frame to Kenya data frame
df = ken_df.append(nig_df)